In [332]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F

import cv2

In [333]:
#load image
c = cv2.imread("coloured.jpg")
m = cv2.imread("binmask.jpg")
#m3 = cv2.merge((m,m,m))

#preprocess image
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#Dataloader
print(m.shape)
coloured = (transform(c)).unsqueeze(0)
mask = (transform(m)).unsqueeze(0)


(64, 64, 3)


In [334]:
mask.shape

torch.Size([1, 3, 64, 64])

In [335]:
class encoder(nn.Module):
        def __init__(self):
                super(encoder, self).__init__()
                self.conv1 = nn.Conv2d(3,64, kernel_size=3)
                self.conv2 = nn.Conv2d(64,32, kernel_size=3)
                self.conv3 = nn.Conv2d(32,16, kernel_size=3)
                
                self.fc1 = nn.Linear(16*58*58, 1024)
                self.fc2 = nn.Linear(1024,512)
                self.fc3 = nn.Linear(512,256)
                
                self.upconv1 = nn.ConvTranspose2d(16, 32, 3, stride=1)
                self.upconv2 = nn.ConvTranspose2d(32, 64, 3, stride=1)
                self.upconv3 = nn.ConvTranspose2d(64, 3, 3, stride=1)
   
        def forward(self,x):
                #Encodings
                x = F.relu(self.conv1(x))
                x = F.relu(self.conv2(x))
                x = F.relu(self.conv3(x))
                #print("X shape", x.shape) # torch.Size([1, 16, 58, 58])
                
                #Style vector
                x_style = torch.flatten(x, 1)
                x_style = F.relu(self.fc1(x_style))
                x_style = F.relu(self.fc2(x_style))
                x_style = F.relu(self.fc3(x_style))
                #print("Final X_Style: ", x_style.shape)

                #Image 
                x_image = F.relu(self.upconv1(x))
                x_image = F.relu(self.upconv2(x_image))
                x_image = F.relu(self.upconv3(x_image))
                #print("X_image shape:", x_image.shape)

                
                return x_style, x_image





In [336]:
class decoder(nn.Module):
        def __init__(self):
              super(decoder, self).__init__()
              #Convert style for concat
              self.fc0 = nn.Linear(256,512)

              #Convert image for concat
              self.conv1 = nn.Conv2d(3,64, kernel_size=3)
              self.conv2 = nn.Conv2d(64,32, kernel_size=3)
              self.conv3 = nn.Conv2d(32,16, kernel_size=3)
              self.fc1 = nn.Linear(16*58*58, 1024)
              self.fc2 = nn.Linear(1024,512)

              #Convert image
              self.fc3 = nn.Linear(1024, 1024)
              self.fc4 = nn.Linear(1024, 1024)

              self.upsample1 = nn.Upsample(scale_factor=2,mode='nearest')
              self.upconv1 = nn.ConvTranspose2d(16, 16, 1, stride=1)
              self.upsample2 = nn.Upsample(scale_factor=2,mode='nearest')
              self.upconv2 = nn.ConvTranspose2d(16, 16, 1, stride=1)              
              self.upsample3 = nn.Upsample(scale_factor=2,mode='nearest')
              self.upconv3 = nn.ConvTranspose2d(16, 3, 1, stride=1)

        def forward(self,x_style,x_image):
              x1 = F.relu(self.fc0(x_style)) # 512 style
                
              x2 = F.relu(self.conv1(x_image))
              x2 = F.relu(self.conv2(x2))
              x2 = F.relu(self.conv3(x2))
              x2 = torch.flatten(x2, 1)
              x2 = F.relu(self.fc1(x2))
              x2 = F.relu(self.fc2(x2))     # 512 image

              x = torch.cat([x1,x2], dim=1) #Linear concatenate, output = [1,1024]

              x = F.relu(self.fc3(x))
              x = F.relu(self.fc4(x))
              x = (torch.reshape(x, (16,8,8))).unsqueeze(dim=0)

              x = self.upsample1(x)
              x = F.relu(self.upconv1(x))
              x = self.upsample2(x)
              x = F.relu(self.upconv2(x))
              x = self.upsample3(x)
              x = F.relu(self.upconv3(x))

              return x

In [337]:
enc = encoder()
dec = decoder()

criterion = nn.MSELoss()
optimizer_enc = optim.Adam(enc.parameters(), lr=0.001)
optimizer_dec = optim.Adam(dec.parameters(), lr=0.001)

In [338]:
#a = torch.randn(1,3,64,64)
enc_style, enc_image = enc(coloured)

output = deca(enc_style, enc_image)
#output.shape

torch.Size([1, 16, 16, 16])


In [339]:

output = deca(enc_style, enc_image)
output.shape

torch.Size([1, 16, 16, 16])


torch.Size([1, 3, 64, 64])

In [340]:
#Convert tensors to images

outputimage = torchvision.transforms.ToPILImage()(output.squeeze(0))
maskimage = torchvision.transforms.ToPILImage()(enc_image.squeeze(0))


In [341]:

for epoch in range(10):
  #Training Encoder
  optimizer_enc.zero_grad() 

  enc_style, enc_mask = enc(coloured)
  loss_mask = criterion(enc_mask, mask)
  loss_mask.backward()
  optimizer_enc.step()

  #Training Decoder
  optimizer_dec.zero_grad() 

  output = dec(enc_style, enc_mask)
  loss_output = criterion(output, coloured)
  loss_output.backward()
  optimizer_dec.step()




RuntimeError: ignored

In [ ]:

criterion = nn.MSELoss()


loss_style = criterion(output,coloured)
loss_mask = criterion(enc_image, mask)

print("Style loss = ", loss_style)
print("Mask loss = ", loss_mask)